In [1]:
from python_workflow_definition.aiida import write_workflow_json, construct_wg_qe
from python_workflow_definition.jobflow import load_workflow_json
from jobflow.managers.local import run_locally

from aiida import load_profile
load_profile()

workflow_json_filename =  "workflow_qe_aiida.json"

/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/paramiko/transport.py:253: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
from quantum_espresso_workflow import (
    generate_structures,
    get_bulk_structure,
    calculate_qe,
    plot_energy_volume_curve,
)

from python_workflow_definition.aiida import pickle_node, construct_wg_qe
from python_workflow_definition.shared import get_list, get_dict

# from python_workflow_definition.pyiron_base import  get_dict
from aiida_workgraph import WorkGraph, task
from typing import Any

load_profile()

get_list_task = task.pythonjob(get_list)
get_dict_task = task.pythonjob(get_dict)
pickle_node_task = task.pythonjob(pickle_node)
get_bulk_structure_task = task.pythonjob()(get_bulk_structure)
generate_structures_task = task.pythonjob()(generate_structures)
calculate_qe_task = task.pythonjob(outputs=["energy", "volume", "structure"])(
    calculate_qe
)
plot_energy_volume_curve_task = task.pythonjob()(plot_energy_volume_curve)

strain_lst = [0.9, 0.95, 1.0, 1.05, 1.1]

wg = construct_wg_qe(
    get_dict_task=get_dict_task,
    get_list_task=get_list_task,
    pickle_node_task=pickle_node_task,
    get_bulk_structure_task=get_bulk_structure_task,
    calculate_qe_task=calculate_qe_task,
    generate_structures_task=generate_structures_task,
    plot_energy_volume_curve_task=plot_energy_volume_curve_task,
    strain_lst=strain_lst
)

wg
wg.to_html("wg-aiida-fix.html")

In [3]:
_ = write_workflow_json(wg=wg, file_name=workflow_json_filename)


In [4]:

flow = load_workflow_json(file_name=workflow_json_filename)

In [5]:
result = run_locally(flow)
result

2025-03-17 13:03:08,254 INFO Started executing jobs locally
2025-03-17 13:03:08,350 INFO Starting job - get_bulk_structure (2752a57d-0f42-46d2-b030-a2061311d315)
2025-03-17 13:03:08,506 INFO Finished job - get_bulk_structure (2752a57d-0f42-46d2-b030-a2061311d315)
2025-03-17 13:03:08,507 INFO Starting job - get_dict (798cca09-0626-4cf6-8236-c5c6f5340458)
> /home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/jobflow/core/reference.py(189)resolve()
    188                 data[attr]
--> 189                 if attr_type == "i" or isinstance(data, dict)
    190                 else getattr(data, attr)

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
2025-03-17 13:04:08,108 INFO get_dict failed with exception:
Traceback (most recent call last):
  File "/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/jobflow/managers/local.py", line 117, in _run_job
    response = job.run(store=store)
  File "/home/geiger_j/.aiida_venvs/adis/lib/python3.10/site-packages/j

{'2752a57d-0f42-46d2-b030-a2061311d315': {1: Response(output={'numbers': [13, 13, 13, 13], 'positions': [[0.0, 0.0, 0.0], [0.0, 2.025, 2.025], [2.025, 0.0, 2.025], [2.025, 2.025, 0.0]], 'cell': [[4.05, 0.0, 0.0], [0.0, 4.05, 0.0], [0.0, 0.0, 4.05]], 'pbc': [True, True, True]}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/geiger_j/aiida_projects/adis/git-repos/python-workflow-definition'))}}